In [ ]:
#For GAM
from googleads import ad_manager
from datetime import datetime, date, timedelta
import tempfile
from googleads import errors
import pandas as pd
from dateutil.relativedelta import relativedelta

#For GSheet
from google.oauth2 import service_account
from googleapiclient.discovery import build

import warnings
warnings.filterwarnings("ignore")

#handle warning default encoder is not UTF-8
import locale
import sys
import _locale

_locale._getdefaultlocale = (lambda *args: ['en_US', 'UTF-8'])

In [ ]:
# inisiasi untuk menentukan credential dan settingan lain
client = ad_manager.AdManagerClient.LoadFromStorage()

# ambil periode last 2 month to date
end_date = datetime.now().date()
start_date = end_date - relativedelta(months=5)
start_date = start_date.replace(day=1)

def gam(query):
        # Initialize a DataDownloader.
        report_downloader = client.GetDataDownloader(version='v202405')

        try:
                print ('\nProcessing Report....')
                # jalankan report, output berupa id report
                report_job_id = report_downloader.WaitForReport(query)
        except errors.AdManagerReportError as e:
                print('Failed to generate report. Error was: %s' % e)

        #simpan file sementara di folder temp komputer
        report_file = tempfile.NamedTemporaryFile(suffix='.csv.gz', delete=False)

        #download id report yg sudah di generate
        report_downloader.DownloadReportToFile(report_job_id, 'CSV_DUMP', report_file)
        report_file.close()

        #masukkan dalam data frame
        try:
                df = pd.read_csv(report_file.name).sort_values(by='Dimension.DATE')
        except:
                df = pd.read_csv(report_file.name)
        return df

#Proses Gsheet
SERVICE_ACCOUNT_FILE = 'H:\\Other computers\\Redmibook 15\\Kompas Gramedia\\Reporting\\Laptop Kantor\\GAM\\credentials.json'

# buat credential dari service account
credentials = service_account.Credentials.from_service_account_file(
        filename=SERVICE_ACCOUNT_FILE
)

#inisiasi fungsi google sheet
service = build('sheets','v4', credentials=credentials)

def update(spreadsheet_id,range,df):
        service.spreadsheets().values().clear(spreadsheetId=spreadsheet_id, range=range , body={}).execute()
        # request google sheet
        service.spreadsheets().values().update(spreadsheetId=spreadsheet_id, range=range, valueInputOption='USER_ENTERED' , body={
                'majorDimension':'ROWS',
                'values': df.values.tolist()
        }).execute()


def lastupdate(report,source):
        df=pd.DataFrame({
                "Report":[report],
                "Time": [str(datetime.now())[:16]]
        })
        update('1ddO4pWH3cPP94d1e3Zh2VaLJwnRjOYLYuUeEbXIH9tc','Report Status!'+source,df)


In [ ]:
query_daily = {
                'reportQuery': {
                        'dimensions': ['LINE_ITEM_ID', 'DATE'],
                        'statement': {'query': "WHERE LINE_ITEM_NAME LIKE '%VCBL%' ", 'values': None},
                        'columns': ['AD_SERVER_IMPRESSIONS', 'AD_SERVER_CLICKS','TOTAL_ACTIVE_VIEW_VIEWABLE_IMPRESSIONS','AD_SERVER_CPM_AND_CPC_REVENUE','VIDEO_VIEWERSHIP_START','VIDEO_VIEWERSHIP_COMPLETE'],
                        'dateRangeType': 'CUSTOM_DATE',
                        'startDate': start_date,
                        'endDate': end_date #cut off yesterday
                }
        }

daily = gam(query_daily)
daily['Column.VIDEO_VIEWERSHIP_START'] = daily['Column.VIDEO_VIEWERSHIP_START'].replace('-',0)
daily['Column.VIDEO_VIEWERSHIP_COMPLETE'] = daily['Column.VIDEO_VIEWERSHIP_COMPLETE'].replace('-',0)

update('1XAi4t4hg3pff1tT2X0Ae0RAc3TnxCrdcHhVHPJ2dkTg','A2:H', daily)
print("Report Daily Sucsess\n"+str(datetime.now()))
lastupdate("DAILY",'A2:B')

In [ ]:
query_sampoerna = {
                'reportQuery': {
                        'dimensions': ['LINE_ITEM_ID','LINE_ITEM_NAME', 'DATE'],
                        'statement': {'query': "WHERE ORDER_ID IN (2913452336, 2914181517, 2915970194, 3066621288, 2992907243, 2913702475, 2913701671, 2915969438, 3270951513, 3401421534, 3529055756,3551912534) ", 'values': None},
                        'columns': ['AD_SERVER_IMPRESSIONS', 'AD_SERVER_CLICKS','TOTAL_ACTIVE_VIEW_VIEWABLE_IMPRESSIONS'],
                        'dateRangeType': 'CUSTOM_DATE',
                        'startDate': (datetime.now().date()- relativedelta(months=3)).replace(day=1),
                        'endDate': datetime.now().date()-timedelta(days=1)
                }
        }

update('1CLdNzQQw8egsQcwf7aRcUc1tilB1TNNrJ6XF6lOdceE','Source!B3:G', gam(query_sampoerna))
print("Report Sampoerna Sucsess\n"+str(datetime.now()))
lastupdate("SAMPOERNA",'A3:B')

In [ ]:
query_goal_sampoerna = {
                'reportQuery': {
                        'dimensions': ['LINE_ITEM_ID'],
                        'dimensionAttributes': ['LINE_ITEM_START_DATE_TIME','LINE_ITEM_END_DATE_TIME','LINE_ITEM_GOAL_QUANTITY'],
                        'statement': {'query': "WHERE ORDER_ID IN  (2913452336, 2914181517, 2915970194, 3066621288, 2992907243, 2913702475, 2913701671, 2915969438, 3270951513, 3401421534, 3529055756, 3248241556,3551912534) ", 'values': None},
                        'columns': ['TOTAL_LINE_ITEM_LEVEL_IMPRESSIONS'],
                        'dateRangeType': 'CUSTOM_DATE',
                        'startDate': (datetime.now().date()- relativedelta(months=3)).replace(day=1),
                        'endDate': datetime.now().date()
                }
        }

goal =gam(query_goal_sampoerna).fillna(0)
update('1CLdNzQQw8egsQcwf7aRcUc1tilB1TNNrJ6XF6lOdceE','Source!BF3:BJ', goal)
print("Report Goal Sampoerna Sucsess\n"+str(datetime.now()))
lastupdate("GOAL SAMPOERNA",'A4:B')

In [ ]:
query_iqos = {
                'reportQuery': {
                        'dimensions': ['LINE_ITEM_ID','LINE_ITEM_NAME', 'DATE','HOUR'],
                        'statement': {'query': "WHERE ORDER_ID =3248241556", 'values': None},
                        'columns': ['AD_SERVER_IMPRESSIONS', 'AD_SERVER_CLICKS'],
                        'dateRangeType': 'CUSTOM_DATE',
                        'startDate': date.today()-timedelta(days=60),
                        'endDate': date.today()-timedelta(days=1)
                }
        }

df =  gam(query_iqos)

# Date + Hour
df['Date'] = df['Dimension.DATE']+" "+df['Dimension.HOUR'].astype(str)+":00:00"

#Buat function untuk konversi timezone
import pytz

def timezone(dt):
        #set timezone
        jakarta = pytz.timezone('Etc/GMT+0')
        london = pytz.timezone('Etc/GMT+7')

        dt = datetime.strptime(dt, "%Y-%m-%d %H:%M:%S") #convert string to datetime
        dt = jakarta.localize(dt) #set timezone awal
        dt = dt.astimezone(london) #convert ke timezone tujuan
        return dt.strftime("%Y-%m-%d") #return dalam bentuk string (tanpa time krn hanya mengambil date saja)


df['Date2'] = df['Date'].apply(lambda x: timezone(x)) #buat kolom Date2 dg tanggal yang sudah terkonversi

#merapikan kolom
df.drop(columns=['Date','Dimension.DATE','Dimension.HOUR'], inplace=True)
df.columns=['Line ID','Name','Impression','Click','Date']
df = df[['Line ID','Name','Date','Impression','Click']]

df = df.groupby(['Line ID','Name','Date']).sum().reset_index() #di pivot karena tadi breakdown by hour

update('1CLdNzQQw8egsQcwf7aRcUc1tilB1TNNrJ6XF6lOdceE','Source!Ag3:AK',df)
print("Report IQOS Sucsess\n"+str(datetime.now()))
lastupdate("IQOS",'A6:B')

In [ ]:
query_all = {
                'reportQuery': {
                        'dimensions': ['LINE_ITEM_ID','LINE_ITEM_NAME'],
                        'dimensionAttributes':['LINE_ITEM_PRIORITY','LINE_ITEM_DELIVERY_PACING','LINE_ITEM_GOAL_QUANTITY','LINE_ITEM_COST_TYPE','LINE_ITEM_DELIVERY_INDICATOR','LINE_ITEM_START_DATE_TIME','LINE_ITEM_END_DATE_TIME'],
                        'statement': {'query': "WHERE LINE_ITEM_NAME LIKE '%VCBL%' ", 'values': None},
                        'columns': ['AD_SERVER_IMPRESSIONS', 'AD_SERVER_CLICKS','TOTAL_ACTIVE_VIEW_VIEWABLE_IMPRESSIONS'],
                        'dateRangeType': 'CUSTOM_DATE',
                        'startDate': (date.today() - relativedelta(months=3)).replace(day=1),
                        'endDate': date.today()
                }
}

df = gam(query_all)

df['DimensionAttribute.LINE_ITEM_END_DATE_TIME'] = df['DimensionAttribute.LINE_ITEM_END_DATE_TIME'].astype('string') #konversi ke string dulu

def convert(string):
        string=string[:10] #ambil 10 huruf dari kiri
        return datetime.strptime(string, "%Y-%m-%d") #jadikan type date

df['DimensionAttribute.LINE_ITEM_END_DATE_TIME'] = df['DimensionAttribute.LINE_ITEM_END_DATE_TIME'].apply(lambda x: convert(x))


df =  df[df['DimensionAttribute.LINE_ITEM_END_DATE_TIME']>=date.today().strftime('%Y-%m-%d')] #filter yg end datenya > today artinya campaign yg masih berjalan


df = df.sort_values(by='Dimension.LINE_ITEM_NAME',ascending=True)

df['DimensionAttribute.LINE_ITEM_END_DATE_TIME'] = df['DimensionAttribute.LINE_ITEM_END_DATE_TIME'].astype('string') #kembalikan ke string agar bisa masuk gsheet

df=df.fillna(0)

update('1WQdjDR9U9h2-Obz5PPqi2rj7ebcJuSgFviIdFP3F-T8','A2:L', df)
print("Report ALL Sucsess\n"+str(datetime.now()))
lastupdate("ALL",'A8:B')

In [ ]:
query_reach = {
                'reportQuery': {
                        'dimensions': ['LINE_ITEM_ID', 'LINE_ITEM_NAME'],
                        'statement': {'query': "WHERE LINE_ITEM_NAME LIKE '%VCBL%' ", 'values': None},
                        'columns': ['UNIQUE_REACH'],
                        'dateRangeType': 'REACH_LIFETIME'
                }
        }

reach = gam(query_reach)
update('1XAi4t4hg3pff1tT2X0Ae0RAc3TnxCrdcHhVHPJ2dkTg','Reach!A2:C', reach)

In [ ]:
query_garda = {
                'reportQuery': {
                        'dimensions': ['LINE_ITEM_ID', 'CREATIVE_NAME', 'DATE'],
                        'statement': {'query': "WHERE LINE_ITEM_ID IN(6719407146,6719430609,6727291607,6735015331)", 'values': None},
                        'columns': ['AD_SERVER_IMPRESSIONS', 'AD_SERVER_CLICKS','TOTAL_ACTIVE_VIEW_VIEWABLE_IMPRESSIONS'],
                        'dateRangeType': 'CUSTOM_DATE',
                        'startDate': start_date,
                        'endDate': end_date #cut off yesterday
                }
        }

garda = gam(query_garda)


update('1gUmL01P_3h5Rmjvmt-tYLC2erGHD1OPwNk60BMNI3v4','source!BI2:BO', garda)
print("Report Garda Sucsess\n"+str(datetime.now()))
lastupdate("DAILY",'A9:B')

In [ ]:
query_la = {
                'reportQuery': {
                        'dimensions': ['LINE_ITEM_ID', 'DATE','CREATIVE_NAME'],
                        'statement': {'query': "WHERE LINE_ITEM_NAME LIKE '%202405311909%' ", 'values': None},
                        'columns': ['AD_SERVER_IMPRESSIONS', 'AD_SERVER_CLICKS','TOTAL_ACTIVE_VIEW_VIEWABLE_IMPRESSIONS'],
                        'dateRangeType': 'CUSTOM_DATE',
                        'startDate': start_date,
                        'endDate': end_date #cut off yesterday
                }
        }

la = gam(query_la)
update('1PueZfRquAmqqIV3GTqHac30Bq7pR7OnfVLU9-QvMG24','Source!AI2:AO', la)